In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
train

In [ ]:
y = train['label']

In [ ]:
X = train.drop('label',axis=1)

In [ ]:
y.value_counts().plot.bar()

In [ ]:
X = X.T

In [ ]:
plt.imshow((X[90].values).reshape(28,28))

In [ ]:
X = np.array([ (X[image].values).reshape(28,28) for image in X.columns])

In [ ]:
plt.imshow(X[0])

In [ ]:
X = X/255

In [ ]:
X = X.reshape((42000, 28, 28,1))

In [ ]:
y = y.values

In [ ]:
y

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=0)

In [ ]:
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',input_shape=(28,28,1)))

In [ ]:
model.add(MaxPooling2D((2,2),(2,2)))

In [ ]:
model.add(Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform'))

In [ ]:
model.add(MaxPooling2D((2,2),(2,2)))

In [ ]:
model.add(Conv2D(128,(3,3),activation='relu',kernel_initializer='he_uniform'))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dropout(0.25))

In [ ]:
model.add(Dense(256,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=regularizers.l2(0.001)))

In [ ]:
model.add(Dense(10,activation='softmax',kernel_initializer='he_uniform'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,epochs=10,validation_split=0.1,steps_per_epoch=34020)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1,  
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False,  
        vertical_flip=False)  


datagen.fit(X)

In [ ]:
model.fit_generator(datagen.flow(X,y),epochs=12,validation_data=(X_test,y_test))

In [ ]:
model.save('digit_recognizer.h5')

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test = test.T

In [ ]:
test

In [ ]:
test = np.array([ (test[image].values).reshape(28,28) for image in test.columns])

In [ ]:
test = test/255

In [ ]:
test = test.reshape((28000, 28, 28,1))

In [ ]:
y_t = pd.Series(model.predict_classes(test))

In [ ]:
sample = pd.read_csv('sample_submission.csv')

In [ ]:
predictions = pd.concat([sample.drop('Label',axis=1),y_t],axis=1)

In [ ]:
predictions.columns = ['ImageId','Label']

In [ ]:
predictions.to_csv('predictions.csv',index=False)